# markdown.obisidian.personal.reference
> Functions for managing references in an Obsidian.md math vault

In [2]:
#| default_exp markdown.obsidian.personal.reference

In [3]:
#| export
import glob
import os
from os import PathLike
from pathlib import Path
import re
import shutil
from typing import Union, Optional
import warnings

from trouver.helper import (
    path_name_no_ext, alphabet_to_alphabet_group
)
from trouver.markdown.markdown.file import (
    MarkdownFile, MarkdownLineEnum
)
from trouver.markdown.markdown.heading import (
    heading_title
)
from trouver.markdown.obsidian.links import (
    ObsidianLink, LinkType, links_from_text
)
from trouver.markdown.obsidian.personal.authors import find_author_file
from trouver.markdown.obsidian.personal.index_notes import ( 
    convert_title_to_folder_name
)
from trouver.markdown.obsidian.personal.notes import (
    notes_linked_in_note
)
from trouver.markdown.obsidian.personal.note_type import (
    type_of_note, PersonalNoteTypeEnum
)
from trouver.markdown.obsidian.vault import(
    VaultNote, all_note_paths_by_name, note_path_by_name,
    NoteDoesNotExistError, NoteNotUniqueError
)

In [12]:
from fastcore.test import *

from trouver.helper import _test_directory

# Getting a reference folder

In [5]:
#| export
def index_note_for_reference(
        vault: PathLike, # The vault in which the reference folder resides.
        reference: Union[str, Path] # - The reference. Is either - a str, in which case the reference folder will be the folder containing the (unique) note of the name `_index_{reference}.md`, - or a `Path` object (not just a pathlike!) relative to `vault`, in which case the path will be the path to the reference folder. 
        ) -> VaultNote:
    """
    Returns the index note of the specified reference in the vault.
    
    Assumes that the reference folder has an index note named
    `_index_{reference_name}.md` and this note is the unique note in the vault
    with this filename.
    
    **Raises**

    - TypeError
        - If `reference` is not a str or PathLike.
    - NoteDoesNotExistError
        - If a note of the name `_index_{reference_name}.md` does not exist
        in the vault.
    """
    if (not isinstance(reference, str)
            and not isinstance(reference, PathLike)):
        raise TypeError(
            "Expected `reference` to be a str or a PathLike, but got"
            f" {type(reference)} instead.")
    if isinstance(reference, str):
        reference_name = reference
        index_note = VaultNote(vault, name=f'_index_{reference_name}')
    elif isinstance(reference, PathLike):
        reference_name = Path(reference).name
        index_note = VaultNote(
            vault, rel_path=Path(reference) / f'_index_{reference_name}.md')
    return index_note



The `index_note_for_reference` method obtains the index note of the specified reference in the vault.

In [10]:
vault = _test_directory() / 'test_vault_5'
index_note_1 = index_note_for_reference(vault, reference='number_theory_reference_1')
assert index_note_1.name == '_index_number_theory_reference_1'

index_note_2 = index_note_for_reference(vault, reference=Path('number_theory') / 'number_theory_reference_1')
assert index_note_2.name == '_index_number_theory_reference_1'

In [20]:
#| export
def reference_directory(
        vault: PathLike, # The vault in which the reference folder resides.
        reference: Union[str, Path] # - The reference. Is either - a str, in which case the reference folder will be the folder containing the (unique) note of the name `_index_{reference}.md`, - or a `Path` object (not just a pathlike!) relative to `vault`, in which case the path will be the path to the reference folder. 
        ) -> Path: # Relative to `vault`.
    """
    Returns the path to the reference directory in a vault.
    
    Assumes that the reference folder has an index note named
    `_index_{reference_name}.md`, this note is the unique note in the vault
    with this filename, and the cache in the `VaultNote` class for `vault` is
    updated.

    **Raises**

    - TypeError
        - If `reference` is not a str or PathLike.
    - 
    
    """
    index_note = index_note_for_reference(vault, reference)
    if index_note.exists(update_cache=False):
        return Path(index_note.path(relative=True)).parent
    else:
        raise NoteDoesNotExistError.from_note_name(index_note.name)

The `reference_directory` method obtains the root directory for the reference:

In [21]:
vault = _test_directory() / 'test_vault_5'
dir = reference_directory(vault, reference='number_theory_reference_1')

If the vault does not have an index note for the specified reference, then a `NoteDoesNotExistError` is raised:

In [22]:
with ExceptionExpected(NoteDoesNotExistError):
    thing = reference_directory(vault, reference='bad_number_theory_reference_without_an_index_note')
    print(thing)